In [1]:
from os.path import dirname, join, isdir
from datetime import datetime
from os import makedirs
import pandas as pd
import numpy as np
from simpledbf import Dbf5
from bulletin import __file__ as __root__
from bulletin.commom import static

pd.set_option('display.max_columns', None)

In [2]:
srag = Dbf5(join('input','dbf','DOPR2020.dbf'), codec = 'cp860').to_dataframe()
srag2 = Dbf5(join('input','dbf','DOPR2021.dbf'), codec = 'cp860').to_dataframe()

In [3]:
srag = srag.append(srag2, ignore_index=True)

In [4]:
srag = srag[['NUMERODO', 'NUMERODV', 'DTOBITO', 'NUMSUS', 'NOME', 'NOMEMAE', 'DTNASC',
            'IDADE', 'SEXO', 'RACACOR', 'OCUP', 'CODESTRES', 'CODMUNRES', 'BAIRES',
            'ENDRES', 'NUMRES', 'CEPRES', 'CODESTOCOR', 'CODMUNOCOR', 'LINHAA', 'LINHAB',
            'LINHAC', 'LINHAD', 'LINHAII', 'CAUSABAS']]

In [5]:
srag = srag.rename(
    columns={
        'DTOBITO': 'data_cura_obito',
        'NUMSUS': 'cns',
        'NOME': 'paciente',
        'NOMEMAE': 'nome_mae',
        'DTNASC': 'data_nascimento',
        'IDADE': 'idade',
        'SEXO': 'sexo',
        'RACACOR': 'raca_cor',
        'OCUP': 'cod_cbo',
        'CODESTRES': 'uf_residencia',
        'CODMUNRES': 'ibge_residencia',
        'BAIRES': 'bairro_residencia',
        'ENDRES': 'logradouro_residencia',
        'NUMRES': 'numero_residencia',
        'CEPRES': 'cep_residencia',
        'CODESTOCOR': 'uf_unidade_notifica',
        'CODMUNOCOR': 'ibge_unidade_notifica'
    })

In [6]:
srag.loc[srag['raca_cor'] == '3', 'raca_cor'] = '-1'
srag.loc[srag['raca_cor'] == '4', 'raca_cor'] = '3'
srag.loc[srag['raca_cor'] == '-1', 'raca_cor'] = '4'
srag.loc[srag['raca_cor'].isnull(), 'raca_cor'] = '99'

In [7]:
srag["data_cura_obito"] = srag["data_cura_obito"].apply(lambda x: pd.to_datetime(x, format='%d%m%Y'))
srag["data_nascimento"] = srag["data_nascimento"].apply(lambda x: pd.to_datetime(x, format='%d%m%Y'))

In [8]:
srag['idade'] = pd.to_numeric(srag['idade'], downcast = 'integer')
srag['idade'] = srag['idade'].apply(lambda x: np.subtract(x, 400))
srag.loc[srag['idade'] < 1, 'idade'] = 0
srag.loc[srag['idade'] == 599, 'idade'] = None

In [9]:
srag.loc[srag['nome_mae'].isnull(), 'nome_mae'] = ''
nao = set(['NAO','CONSTA','INFO','INFORMADO','CONTEM',''])
srag.loc[ [True if set(nome_mae.split(" ")).intersection(nao) else False for nome_mae in srag['nome_mae'] ], 'nome_mae'] = None

In [10]:
srag.loc[srag['cns'].isnull(), 'cns'] = '0'
srag.loc[srag['cod_cbo'].isnull(), 'cod_cbo'] = '0'
srag.loc[srag['numero_residencia'].isnull(), 'numero_residencia'] = '0'
srag.loc[srag['cep_residencia'].isnull(), 'cep_residencia'] = '0'

In [11]:
srag['id'] = srag['NUMERODO'].str.cat(srag['NUMERODV'])

In [12]:
srag['LINHAA'] = srag['LINHAA'].astype('string')
srag.loc[srag['LINHAA'].isnull(), 'LINHAA'] = ''
srag.loc[srag['LINHAB'].isnull(), 'LINHAB'] = ''
srag.loc[srag['LINHAC'].isnull(), 'LINHAC'] = ''
srag.loc[srag['LINHAD'].isnull(), 'LINHAD'] = ''
srag.loc[srag['LINHAII'].isnull(), 'LINHAII'] = ''
srag.loc[srag['CAUSABAS'].isnull(), 'CAUSABAS'] = ''
srag.loc[((srag['LINHAB'].apply(lambda x: 'B342' in x)) | (srag['LINHAC'].apply(lambda x: 'B342' in x)) |
        (srag['LINHAD'].apply(lambda x: 'B342' in x)) | (srag['LINHAII'].apply(lambda x: 'B342' in x)) |
        (srag['CAUSABAS'].apply(lambda x: 'B342' in x))), 'evolucao'] = '2'
srag.loc[srag['evolucao'] != '2', 'evolucao'] = '4'

In [13]:
srag = srag[['id', 'data_cura_obito', 'cns', 'paciente', 'nome_mae', 'data_nascimento', 'idade', 'sexo',
            'raca_cor', 'cod_cbo', 'uf_residencia', 'ibge_residencia', 'bairro_residencia', 'logradouro_residencia',
            'cep_residencia', 'uf_unidade_notifica', 'ibge_unidade_notifica', 'evolucao']]

In [14]:
srag['sistema'] = 'SIM'